# Word2vec & Fasttext

Есть 2 режима обучения word2vec:
- cbow
- skipgram


<img src='image/w2v.png'>


Fasttext работает +- также как word2vec, только включает в вектор слова еще n-gramm побуквенные.

<img src='image/ft.png'>


Важно помнить, что энграммы хешируются -> маленькое число бакетов -> просад по качеству

# Тематическое моделирование

<img src='image/tm.webp'>


In [1]:
import re
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords


df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


In [2]:
data = df.content.values.tolist()

data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

In [3]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data))
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [10]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [11]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)

In [12]:
id2word = corpora.Dictionary(data_words_bigrams)
texts = data_words_bigrams
corpus = [id2word.doc2bow(text) for text in texts]

In [13]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=1,
                                           alpha='auto',
                                           per_word_topics=True)

In [14]:
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.079*"jesus" + 0.079*"faith" + 0.079*"god" + 0.064*"christians" + 0.059*"bible" + 0.045*"christian" + 0.038*"church" + 0.029*"christ" + 0.014*"indeed" + 0.013*"jewish"'), (1, '0.050*"newsreader_tin" + 0.042*"self" + 0.039*"la" + 0.039*"version_pl" + 0.034*"motif" + 0.031*"al" + 0.017*"blue" + 0.017*"helmet" + 0.016*"activity" + 0.014*"levels"'), (2, '0.010*"time" + 0.009*"us" + 0.008*"may" + 0.008*"first" + 0.007*"two" + 0.007*"help" + 0.006*"day" + 0.006*"said" + 0.005*"new" + 0.005*"one"'), (3, '0.060*"senses" + 0.031*"bike" + 0.026*"speak" + 0.024*"engine" + 0.019*"dod" + 0.018*"power" + 0.017*"picture" + 0.015*"stolen" + 0.015*"disagree" + 0.011*"owner"'), (4, '0.112*"physical" + 0.031*"dc" + 0.031*"direct" + 0.021*"ac" + 0.018*"oo" + 0.018*"ab" + 0.018*"gov" + 0.017*"spacecraft" + 0.016*"vms_vnews" + 0.016*"software_vax"'), (5, '0.050*"team" + 0.045*"game" + 0.036*"sale" + 0.030*"year" + 0.027*"games" + 0.026*"points" + 0.023*"win" + 0.022*"teams" + 0.021*"color" + 0.021*"p

## Тема 3. Разметка и извлечение именованных сущностей

### 3.1. Разметка слов с помощью частей речи (Parts-Of-Speech)

<img src='image/pos_tagging.PNG'>

POS разметка - определение части речи и грамматических характеристик слов в тексте (корпусе) с приписыванием им соответствующих тегов. POS tagging является одним из первых этапов компьютерного анализа текста. Эта задача не простая, так как конкретное слово может иметь различную часть речи в зависимости от контекста, в котором оно используется.
Например: в предложении “Дай мне свой ответ” ответ - это существительное, а в предложении “ответь на вопрос” ответ - это глагол.

POS разметка полезна для построения деревьев синтаксического анализа, которые используются при построении NER (именованных сущностей) и извлечении отношений между словами. POS-маркировка также необходима для построения лемматизаторов.

In [16]:
# nltk.download() 
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline

In [17]:
import warnings
warnings.filterwarnings("ignore")

#### Универсальный набор тегов части речи

Ниже приведена таблица POS-тэгов: обозначение, часть речи, пример. Цветом выделены популярные части речи: существительное, глагол и вопросительное слово.

<img src='image/pos_tag_samples.jpg'>

Выведим примеры тэгов разных частей речи.

In [19]:
nltk.download('tagsets')
nltk.help.upenn_tagset('RB')
nltk.help.upenn_tagset('NN')
nltk.help.upenn_tagset('VB')

[nltk_data] Downloading package tagsets to /Users/radik/nltk_data...


RB: adverb
    occasionally unabatingly maddeningly adventurously professedly
    stirringly prominently technologically magisterially predominately
    swiftly fiscally pitilessly ...
NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
VB: verb, base form
    ask assemble assess assign assume atone attention avoid bake balkanize
    bank begin behold believe bend benefit bevel beware bless boil bomb
    boost brace break bring broil brush build ...


[nltk_data]   Unzipping help/tagsets.zip.


#### Сравнение POS тэггеров

POS-тэггер обрабатывает последовательность слов и определяет тэг части речи для каждого слова. Сравним работу нескольких тэггеров библиотеки nltk.tag. Проверять работоспособность теггеров будем на корпусе nltk.corpus.brown.

Отобразим распределение тэгов в корпусе brown. Можем видеть, что тэг "NN" наиболее популярный.

Корпус будет поделен на train и test, т.к. некоторым тэггерам необходимо обучение. Разметку визуально будем оценивать на примере test_sent, тестового предложения.

In [21]:
nltk.download('brown')
from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger
from nltk.tag import RegexpTagger

tags = [tag for (word, tag) in
brown.tagged_words(categories='news')]
nltk.FreqDist(tags)

[nltk_data] Downloading package brown to /Users/radik/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


FreqDist({'NN': 13162, 'IN': 10616, 'AT': 8893, 'NP': 6866, ',': 5133, 'NNS': 5066, '.': 4452, 'JJ': 4392, 'CC': 2664, 'VBD': 2524, ...})

In [22]:
brown_tagged_sents = brown.tagged_sents(categories='news')
train_data = brown_tagged_sents[:int(len(brown_tagged_sents) * 0.9)]
test_data = brown_tagged_sents[int(len(brown_tagged_sents) * 0.9):]
test_sent = brown.sents(categories='news')[0]
test_data[0]

[('But', 'CC'),
 ('in', 'IN'),
 ('all', 'ABN'),
 ('its', 'PP$'),
 ('175', 'CD'),
 ('years', 'NNS'),
 (',', ','),
 ('not', '*'),
 ('a', 'AT'),
 ('single', 'AP'),
 ('Negro', 'NP'),
 ('student', 'NN'),
 ('has', 'HVZ'),
 ('entered', 'VBN'),
 ('its', 'PP$'),
 ('classrooms', 'NNS'),
 ('.', '.')]

#### DefaultTagger

Очень наивный тэггер, в данном случае присваивает тэг "NN" (noun) всем словам в тексте. Т.к. в английском тексте примерно 13% сущесвительных, то получим точность тэггирования примерно 0,13.

У каждого теггера есть метод .tag(), который принимает список токенов (обычно список слов, созданных токенизатором слов), где каждый токен-это одно слово.
Метод .evaluate() оценивает точность работы тэггера.

In [23]:
default_tagger = nltk.DefaultTagger('NN')
display(default_tagger.tag(test_sent), default_tagger.evaluate(test_data))

[('The', 'NN'),
 ('Fulton', 'NN'),
 ('County', 'NN'),
 ('Grand', 'NN'),
 ('Jury', 'NN'),
 ('said', 'NN'),
 ('Friday', 'NN'),
 ('an', 'NN'),
 ('investigation', 'NN'),
 ('of', 'NN'),
 ("Atlanta's", 'NN'),
 ('recent', 'NN'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'NN'),
 ('``', 'NN'),
 ('no', 'NN'),
 ('evidence', 'NN'),
 ("''", 'NN'),
 ('that', 'NN'),
 ('any', 'NN'),
 ('irregularities', 'NN'),
 ('took', 'NN'),
 ('place', 'NN'),
 ('.', 'NN')]

0.1262832652247583

#### UnigramTagger

UnigramTagger учитывает условную частоту тегов и предсказывает наиболее частый тег для каждого токена, не ориентируется на соседние слова.

In [24]:
unigram_tagger = UnigramTagger(train_data)
display(unigram_tagger.tag(test_sent), unigram_tagger.evaluate(test_data))

[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]

0.8121200039868434

#### BigramTagger

BigramTagger будет учитывает тэги двух слов: текущее и предыдущее слово. Точность немного выше, чем у Unigram tagger.

In [25]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
display(bigram_tagger.tag(test_sent), bigram_tagger.evaluate(test_data))

[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]

0.8210904016744742

#### RegexpTagger

RegexpTagger работает на основе поиска совпаднения с регулярными выражениями. Пример: числа можно сопоставить с регулярным выражением "\d" для присвоения тега CD (Cardinal number) или можно сопоставить известные шаблоны слов, такие как суффикс '.*ing$'.

In [26]:
patterns = [
    (r'.*ing$', 'VBG'),                # gerunds
    (r'.*ed$', 'VBD'),                 # simple past
    (r'.*es$', 'VBZ'),                 # 3rd singular present
    (r'.*ould$', 'MD'),                # modals
    (r'.*\'s$', 'NN$'),                # possessive nouns
    (r'.*s$', 'NNS'),                  # plural nouns
    (r'^-?[0-9]+(\.[0-9]+)?$', 'CD'),  # cardinal numbers
    (r'.*', 'NN'),                      # nouns (default) 
    (r'.*ment$', 'NN'),                # i.e. wonderment 
    (r'.*ful$', 'JJ')                  # i.e. wonderful 
]
regexp_tagger = RegexpTagger(patterns)
display(regexp_tagger.tag(test_sent), regexp_tagger.evaluate(test_data))

[('The', 'NN'),
 ('Fulton', 'NN'),
 ('County', 'NN'),
 ('Grand', 'NN'),
 ('Jury', 'NN'),
 ('said', 'NN'),
 ('Friday', 'NN'),
 ('an', 'NN'),
 ('investigation', 'NN'),
 ('of', 'NN'),
 ("Atlanta's", 'NN$'),
 ('recent', 'NN'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', 'NN'),
 ('no', 'NN'),
 ('evidence', 'NN'),
 ("''", 'NN'),
 ('that', 'NN'),
 ('any', 'NN'),
 ('irregularities', 'VBZ'),
 ('took', 'NN'),
 ('place', 'NN'),
 ('.', 'NN')]

0.20253164556962025

Судя по результату, регулярных выражений в patterns не достаточно, чтобы покрыть большой процент слов корпуса.

В дополнение используем pos_tag тэггер и отобразим результат разметки в виде структуры "Tree"(дерево).

In [28]:
nltk.download('averaged_perceptron_tagger')
from nltk.chunk import ne_chunk
from nltk.tag import pos_tag

sentence = ' '.join(test_sent)
sent_tags = pos_tag(word_tokenize(sentence))
sent_tags [:5]

pattern = 'NP: {<DT>?<JJ>*<NN>}'
NPChunker = nltk.RegexpParser(pattern) 
result = NPChunker.parse(sent_tags)
result.draw()


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/radik/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Фрагмент дерева представлен ниже

<img src='image/draw_tagger.PNG'>

In [29]:
print(result, '\n\n', type(result))

(S
  The/DT
  Fulton/NNP
  County/NNP
  Grand/NNP
  Jury/NNP
  said/VBD
  Friday/NNP
  (NP an/DT investigation/NN)
  of/IN
  Atlanta/NNP
  's/POS
  (NP recent/JJ primary/JJ election/NN)
  produced/VBD
  ``/``
  (NP no/DT evidence/NN)
  ``/``
  that/IN
  any/DT
  irregularities/NNS
  took/VBD
  (NP place/NN)
  ./.) 

 <class 'nltk.tree.Tree'>


#### Комбинация тэггеров

Примущество Backoff Tagging в том, что если текущий тэггер не знает, как тэггировать слово, он передает это следующему и так далее, пока не пройдет перебор по всем тэггерам. В данному случае тэггирование производит последовательность UnigramTagger, BigramTagger, TrigramTagger. Комбинация тэггеров дала немного лучший результат, чем UnigramTagger, BigramTagger по отдельности.

In [30]:
train_data

[[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')], [('The', 'AT'), ('jury', 'NN'), ('further', 'RBR'), ('said', 'VBD'), ('in', 'IN'), ('term-end', 'NN'), ('presentments', 'NNS'), ('that', 'CS'), ('the', 'AT'), ('City', 'NN-TL'), ('Executive', 'JJ-TL'), ('Committee', 'NN-TL'), (',', ','), ('which', 'WDT'), ('had', 'HVD'), ('over-all', 'JJ'), ('charge', 'NN'), ('of', 'IN'), ('the', 'AT'), ('election', 'NN'), (',', ','), ('``', '``'), ('deserves', 'VBZ'), ('the', 'AT'), ('praise', 'NN'), ('and', 'CC'), ('thanks', 'NNS'), ('of', 'IN'), ('the', 'AT'), ('City', 'NN-TL'), ('of', 'IN-TL'), ('Atlant

In [31]:
from nltk.tag import TrigramTagger 

def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(train_data,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(test_data) 

0.843317053722715

#### Создание тэггера имен 

Подходим к теме именнованых сущностей (NER). Мы можем самостоятельно написать необходимый тэггер. Допустим, мы хотим тэггировать имена людей в тексте. Для этого берем корпус имен (nltk.corpus.names), создаем класс тэггера, задаем логику: если слово входит в множество имен, присваиваем ему тэг имени "NNP", иначе тэг "None".

In [33]:
nltk.download('names')
from nltk.tag import SequentialBackoffTagger
from nltk.corpus import names

class NamesTagger(SequentialBackoffTagger):
    def __init__(self, *args, **kwargs):
        SequentialBackoffTagger.__init__(self, *args, **kwargs)
        self.name_set = set([n.lower() for n in names.words()])
            
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word.lower() in self.name_set:
             return 'NNP'
        else:
             return None
            
nt = NamesTagger()
print(nt.tag(['Katya'])) 
print(nt.tag(['Adam'])) 
print(nt.tag(['Window']))            

[nltk_data] Downloading package names to /Users/radik/nltk_data...


[('Katya', 'NNP')]
[('Adam', 'NNP')]
[('Window', None)]


[nltk_data]   Unzipping corpora/names.zip.


Разметка предложений

На вход тэггеру также можно подавать предложения, self.tag() будет применяться к каждому слову предложения.

In [34]:
bigram_tagger.tag_sents([['make', 'America', 'great', 'again'], ['winter', 'is', 'coming']])

[[('make', 'VB'), ('America', 'NP-TL'), ('great', 'JJ'), ('again', 'RB')],
 [('winter', 'NN'), ('is', 'BEZ'), ('coming', 'VBG')]]

####  Разметка корпусов

Некоторые из корпусов, включенных в NLTK, были уже размечены. Вот пример того, что вы можете увидеть, если откроете nltk.corpus.brown

In [35]:
nltk.corpus.brown.tagged_words()

[('The', 'AT'), ('Fulton', 'NP-TL'), ...]

In [37]:
nltk.download('universal_tagset')
nltk.corpus.brown.tagged_words(tagset='universal')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/radik/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


[('The', 'DET'), ('Fulton', 'NOUN'), ...]

Частота частей речи в корпусе

In [38]:
brown_news_tagged = nltk.corpus.brown.tagged_words(categories='adventure', tagset='universal')
tag_fd = nltk.FreqDist(tag for (word, tag) in brown_news_tagged)
tag_fd.most_common()

[('NOUN', 13354),
 ('VERB', 12274),
 ('.', 10929),
 ('DET', 8155),
 ('ADP', 7069),
 ('PRON', 5205),
 ('ADV', 3879),
 ('ADJ', 3364),
 ('PRT', 2436),
 ('CONJ', 2173),
 ('NUM', 466),
 ('X', 38)]

### 3.4.  Извлечение именованных сущностей (NER) и отношений

Одна из самых популярных задач NLP – извлечении именованных сущностей (Named-entity recognition, NER). Задача NER – выделить спаны сущностей в тексте (спан – непрерывный фрагмент текста). Допустим, есть новостной текст, и мы хотим выделить в нем сущности (некоторый заранее зафиксированный набор — например, персоны, локации, организации, даты и так далее). Задача NER – понять, что участок текста “1 января 1997 года” является датой, “Кофи Аннан” – персоной, а “ООН” – организацией.

##### Зачем?
1.  Cтруктуризация неструктурированных данных. Пусть у вас есть какой-то текст (или набор текстов), и данные из него нужно ввести в базу данных (таблицу). 
2. Шаг в сторону “понимания” текста. Это может как иметь самостоятельную ценность, так и помочь лучше решать другие задачи NLP. Без решения задачи NER тяжело представить себе решение многих задач NLP, допустим, разрешение местоименной анафоры или построение вопросно-ответных систем. 

Пример 1: Местоименная анафора позволяет нам понять, к какому элементу текста относится местоимение. Например, пусть мы хотим проанализировать текст “Прискакал Чарминг на белом коне. Принцесса выбежала ему навстречу и поцеловала его”. Если мы выделили на слове “Чарминг” сущность Персона, то машина сможет намного легче понять, что принцесса, скорее всего, поцеловала не коня, а принца Чарминга.

Пример 2: Теперь приведем пример, как выделение именованных сущностей может помочь при построении вопросно-ответных систем. Если задать в вашем любимом поисковике вопрос «Кто играл роль Дарта Вейдера в фильме “Империя наносит ответный удар”», то с большой вероятностью вы получите верный ответ. Это делается как раз с помощью выделения именованных сущностей: выделяем сущности (фильм, роль и т. п.), понимаем, что нас спрашивают, и дальше ищем ответ в базе данных.

Для решения NER-задач существуют множество инструментов. Рассмотрим несколько из них.


#### NLTK

<img src='image/nltk.PNG'>

Для разметки NER с помощью NLTK сначала производим токенизацию слов, затем POS тэггинг. 
В качестве документа возьмем статью 'https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news, распарсим ее с помощью BeautifulSoup.

In [39]:
import requests
from bs4 import BeautifulSoup
import re

def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

document = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')

nltk.pos_tag(nltk.word_tokenize(document))

[('F.B.I', 'NNP'),
 ('.', '.'),
 ('Agent', 'NNP'),
 ('Peter', 'NNP'),
 ('Strzok', 'NNP'),
 (',', ','),
 ('Who', 'NNP'),
 ('Criticized', 'NNP'),
 ('Trump', 'NNP'),
 ('in', 'IN'),
 ('Texts', 'NNP'),
 (',', ','),
 ('Is', 'NNP'),
 ('Fired', 'NNP'),
 ('-', ':'),
 ('The', 'DT'),
 ('New', 'NNP'),
 ('York', 'NNP'),
 ('Times', 'NNP'),
 ('SectionsSEARCHSkip', 'NNP'),
 ('to', 'TO'),
 ('contentSkip', 'VB'),
 ('to', 'TO'),
 ('site', 'NN'),
 ('indexPoliticsToday', 'JJ'),
 ('’', 'NNP'),
 ('s', 'NN'),
 ('PaperPolitics|F.B.I', 'NNP'),
 ('.', '.'),
 ('Agent', 'NNP'),
 ('Peter', 'NNP'),
 ('Strzok', 'NNP'),
 (',', ','),
 ('Who', 'NNP'),
 ('Criticized', 'NNP'),
 ('Trump', 'NNP'),
 ('in', 'IN'),
 ('Texts', 'NNP'),
 (',', ','),
 ('Is', 'VBZ'),
 ('Firedhttps', 'NNP'),
 (':', ':'),
 ('//nyti.ms/2OtNre3AdvertisementContinue', 'NN'),
 ('reading', 'VBG'),
 ('the', 'DT'),
 ('main', 'JJ'),
 ('storySupported', 'JJ'),
 ('byContinue', 'NN'),
 ('reading', 'VBG'),
 ('the', 'DT'),
 ('main', 'JJ'),
 ('storyF.B.I', 'NN'),


С помощью функции nltk.ne_chunk () мы можем распознавать именованные сущности с помощью классификатора, который добавляет метки категорий, такие как PERSON, ORGANIZATION и GPE.

In [43]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/radik/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/radik/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


{('Agent Peter Strzok', 'PERSON'),
 ('Andrew G. McCabe', 'PERSON'),
 ('Anthony D. Weiner', 'PERSON'),
 ('Army', 'ORGANIZATION'),
 ('China', 'GPE'),
 ('Christopher', 'PERSON'),
 ('Clinton', 'PERSON'),
 ('CompanyNYTCoContact', 'ORGANIZATION'),
 ('Congress', 'ORGANIZATION'),
 ('David Bowdich', 'PERSON'),
 ('Director Wray', 'PERSON'),
 ('F.B.I.', 'ORGANIZATION'),
 ('Georgetown University', 'ORGANIZATION'),
 ('Goelman', 'PERSON'),
 ('Hillary', 'ORGANIZATION'),
 ('Hillary Clinton', 'PERSON'),
 ('Horowitz', 'PERSON'),
 ('House', 'ORGANIZATION'),
 ('Hundreds', 'PERSON'),
 ('IndexSite Information', 'ORGANIZATION'),
 ('Is', 'PERSON'),
 ('Is Fired', 'PERSON'),
 ('James B. Comey', 'PERSON'),
 ('Lisa Page', 'PERSON'),
 ('March', 'GPE'),
 ('Michael E. Horowitz', 'PERSON'),
 ('Michael S. SchmidtAug', 'PERSON'),
 ('Midyear Exam', 'PERSON'),
 ('Mr.', 'PERSON'),
 ('Mr. Bowdich', 'PERSON'),
 ('Mr. Goelman', 'PERSON'),
 ('Mr. Horowitz', 'PERSON'),
 ('Mr. McCabe', 'PERSON'),
 ('Mr. Mueller', 'PERSON'),
 ('

#### Spacy

Spacy значительно быстрее NLTK, так как она написана на Cython и работает с объектами.
Для NER Spacy работает как простой классификатор (неглубокая нейронная сеть с одним скрытым слоем). Объект Doc хранит последовательности токенов и все их аннотации.

<img src='image/spacy.PNG'>

In [39]:
# !python -m spacy download en
# !python -m spacy download en_core_web_sm
# !pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

In [35]:
from bs4 import BeautifulSoup
import requests
import re

def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

In [36]:
# !pip3 install -U spacy
# !python -m spacy info
import spacy
from spacy import displacy
# import en_core_web_md

# nlp = en_core_web_md.load()
nlp = spacy.load("en")
ny_bb = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')


#### Deeppavlov

DeepPavlov-это библиотека ИИ с открытым исходным кодом, построенная на TensorFlow и Keras. DeepPavlov предназначена для разработки чат-ботов и сложных разговорных систем, исследований в области nlp и, в частности, диалоговых систем.

DeepPavlov использует несколько более новый вариант глубокой нейронной архитектуры Flair, известный как гибридная модель Bi-LSTM-CRF.

<img src='image/deeppavlov.PNG'>

Используем предтренированную модель "ner_ontonotes_bert_mult", которая работает с разными языками, в том числе и русским. Подадим на распознование предложение на русском языке. 

In [41]:
# !python -m venv env 
# # .\env\Scripts\activate.bat
# !pip3 install deeppavlov --ignore-installed ruamel-yaml
# !python -m deeppavlov install squad_bert
# !python -m deeppavlov install ner_ontonotes

In [19]:
import deeppavlov
from deeppavlov import configs, build_model
deeppavlov_ner = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
deeppavlov_ner([rus_document])

2020-09-05 11:59:25.341 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/ner_ontonotes_bert_mult_v1.tar.gz to /Users/radik/.deeppavlov/ner_ontonotes_bert_mult_v1.tar.gz
100%|██████████| 1.32G/1.32G [16:43<00:00, 1.32MB/s] 
2020-09-05 12:16:08.728 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /Users/radik/.deeppavlov/ner_ontonotes_bert_mult_v1.tar.gz archive into /Users/radik/.deeppavlov/models
2020-09-05 12:16:23.484 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip to /Users/radik/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip
100%|██████████| 663M/663M [07:05<00:00, 1.56MB/s]  
2020-09-05 12:23:29.389 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /Users/radik/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip archive into /Users/radik/.deeppavlov/downloads

2020-09-05 12:24:11.184 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /Users/radik/.deeppavlov/models/ner_ontonotes_bert_mult/tag.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use sta

2020-09-05 12:24:45.764 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /Users/radik/.deeppavlov/models/ner_ontonotes_bert_mult/model]



INFO:tensorflow:Restoring parameters from /Users/radik/.deeppavlov/models/ner_ontonotes_bert_mult/model


[[['Нью',
   '-',
   'Йорк',
   ',',
   'США',
   ',',
   '30',
   'апреля',
   '2020',
   ',',
   '01',
   ':',
   '01',
   '—',
   'REGNUM',
   'В',
   'администрации',
   'президента',
   'США',
   'Дональда',
   'Трампа',
   'планируют',
   'пройти',
   'все',
   'этапы',
   'создания',
   'вакцины',
   'от',
   'коронавируса',
   'в',
   'ускоренном',
   'темпе',
   'и',
   'выпустить',
   '100',
   'млн',
   'доз',
   'до',
   'конца',
   '2020',
   'года',
   ',',
   'передаёт',
   'агентство',
   'Bloomberg',
   'со',
   'ссылкой',
   'на',
   'осведомлённые',
   'источники']],
 [['B-GPE',
   'I-GPE',
   'I-GPE',
   'O',
   'B-GPE',
   'O',
   'B-DATE',
   'I-DATE',
   'I-DATE',
   'O',
   'B-TIME',
   'I-TIME',
   'I-TIME',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-GPE',
   'B-PERSON',
   'I-PERSON',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-MONEY',
   'I-MONEY',
   'I-MONEY',
   'O',
   'B-DATE',
   'I-DA

### 3.4. Извлечение отношений

Можно извлекать не только именнованные сущности (NER), но и отношения между словами в предложении (Relation extraction).

#### Spacy

В уже знакомой нам библеотеке Spacy используем встроенные displaCy визуализатор со style="dep" для отображения отношений.

In [ ]:
# !pip3 install -U spacy
# !python -m spacy info
import spacy
from spacy import displacy
# import en_core_web_md

# nlp = en_core_web_md.load()
nlp = spacy.load("en")

In [43]:
import spacy
from spacy import displacy
#install spacy model
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.2.0/en_core_web_md-2.2.0.tar.gz
# import en_core_web_md
# nlp = en_core_web_md.load()
nlp = spacy.load("en")
doc = nlp("I think Barack Obama met founder of Facebook at occasion of a release of a new NLP algorithm.")

displacy.render(doc, style="dep") # (1)
displacy.render(doc, style="ent") # (2)

#### NLTK

Распарсим документ nltk.corpus.ieer.parsed_docs('NYT_19980315'), extract_rels позволяет нам выделять необходимые отношения, в данном случае мы ищем пары сущностей <'ORG', 'LOC'>. Также мы указали, что текст должен подходить под регулярное выражение IN.

In [45]:
import re
import nltk
nltk.download('ieer')
IN = re.compile(r'.*\bin\b(?!\b.+ing)')
for doc in nltk.corpus.ieer.parsed_docs('NYT_19980315'):
     for rel in nltk.sem.extract_rels('ORG', 'LOC', doc,
                                      corpus='ieer', pattern = IN):
            print(nltk.sem.rtuple(rel))

[nltk_data] Downloading package ieer to /Users/radik/nltk_data...


[ORG: 'WHYY'] 'in' [LOC: 'Philadelphia']
[ORG: 'McGlashan &AMP; Sarrail'] 'firm in' [LOC: 'San Mateo']
[ORG: 'Freedom Forum'] 'in' [LOC: 'Arlington']
[ORG: 'Brookings Institution'] ', the research group in' [LOC: 'Washington']
[ORG: 'Idealab'] ', a self-described business incubator based in' [LOC: 'Los Angeles']
[ORG: 'Open Text'] ', based in' [LOC: 'Waterloo']
[ORG: 'WGBH'] 'in' [LOC: 'Boston']
[ORG: 'Bastille Opera'] 'in' [LOC: 'Paris']
[ORG: 'Omnicom'] 'in' [LOC: 'New York']
[ORG: 'DDB Needham'] 'in' [LOC: 'New York']
[ORG: 'Kaplan Thaler Group'] 'in' [LOC: 'New York']
[ORG: 'BBDO South'] 'in' [LOC: 'Atlanta']
[ORG: 'Georgia-Pacific'] 'in' [LOC: 'Atlanta']


[nltk_data]   Unzipping corpora/ieer.zip.


#### Allennlp

Попробовать извлекать отношения онлайн можно с помощью демо Allennlp  https://demo.allennlp.org/dependency-parsing/. 
Allen NLP предлагает две модели NER с различными архитектурами: Gated Recurrent Unit (GRU) Network, bi-LSTM-CRF model.

<img src='image/allennlp.PNG'>

In [47]:
# !pip install natasha

In [49]:
from pymorphy2 import MorphAnalyzer

In [50]:
morpher = MorphAnalyzer()

In [51]:
morpher.parse("кошка")[0].tag

OpencorporaTag('NOUN,anim,femn sing,nomn')

# Домашнее задание

In [64]:
import pickle

# for manipulating dataframes
import pandas as pd
# for natural language processing: named entity recognition
import spacy
from collections import Counter
import en_core_web_sm
# nlp = en_core_web_sm.load()
# for visualizations
%matplotlib inline

In [54]:
filename = 'output_tweets'
infile = open(filename,'rb')
combine_df = pickle.load(infile)
infile.close()

In [56]:
combine_df

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, cause, offer, wheel..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"
...,...,...,...,...,...,...,...,...
49154,49155,NaN,thought factory: left-right polarisation! #tru...,thought factory left right polarisation trump ...,"[thought, factory, left, right, polarisation, ...","[thought, factory, left, right, polarisation, ...","[thought, factori, left, right, polaris, trump...","[think, factory, leave, right, polarisation, t..."
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...,feeling like mermaid hairflip neverready forma...,"[feeling, like, mermaid, hairflip, neverready,...","[feeling, like, mermaid, hairflip, neverready,...","[feel, like, mermaid, hairflip, neverreadi, fo...","[feel, like, mermaid, hairflip, neverready, fo..."
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...,hillary campaigned today in ohio omg used word...,"[hillary, campaigned, today, in, ohio, omg, us...","[hillary, campaigned, today, ohio, omg, used, ...","[hillari, campaign, today, ohio, omg, use, wor...","[hillary, campaign, today, ohio, omg, use, wor..."
49157,49158,NaN,"happy, at work conference: right mindset leads...",happy at work conference right mindset leads t...,"[happy, at, work, conference, right, mindset, ...","[happy, work, conference, right, mindset, lead...","[happi, work, confer, right, mindset, lead, cu...","[happy, work, conference, right, mindset, lead..."


Используем предобработанные в рамках 1-ого домашнего задания датасет combine_df_prepocessed.pkl. Используем столбец 'clean_tweet'.!

Задание 1.
Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. 

In [61]:
# !pip3 install -U spacy
# !python -m spacy info
import spacy
from spacy import displacy
# import en_core_web_md

# nlp = en_core_web_md.load()
nlp = spacy.load("en")
# ny_bb = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')
nlp.max_length = 3700000
tokens = nlp(''.join(str(combine_df['clean_tweet'].tolist())))
# displacy.render(tokens, jupyter=True, style='ent')


In [72]:
# tokens
# for ent in tokens.ents:
#   print(ent.text,ent.label_)

In [67]:
items = [x.label_ for x in tokens.ents]
Counter(items).most_common(20)

[('DATE', 9729),
 ('PERSON', 7560),
 ('GPE', 4494),
 ('ORG', 4054),
 ('TIME', 1861),
 ('NORP', 1696),
 ('CARDINAL', 1111),
 ('ORDINAL', 706),
 ('FAC', 341),
 ('LOC', 226),
 ('EVENT', 155),
 ('PRODUCT', 89),
 ('LANGUAGE', 45),
 ('QUANTITY', 16),
 ('LAW', 15),
 ('MONEY', 8),
 ('WORK_OF_ART', 6),
 ('PERCENT', 4)]

Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)

DATE

С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. 

In [71]:
items = [x.text for x in tokens.ents if x.label_=='PERSON' or x.label_=='ORG']
Counter(items).most_common(20)

[('ur', 483),
 ('sta', 162),
 ('polar bear', 94),
 ('gop', 77),
 ('hu', 68),
 ('yo', 60),
 ('hillary', 57),
 ('nba', 40),
 ('forex', 37),
 ('bihday', 36),
 ('blur sun', 36),
 ('ripchristina', 34),
 ('cnn', 29),
 ('house', 29),
 ('islam', 29),
 ("gold forex'", 28),
 ('euro', 28),
 ('eu', 27),
 ('congress', 23),
 ('fed', 23)]

Действительно ли в топ вошли только персоны и организации или есть мусор?

In [ ]:
Действительно, откуда-то взялся мусор

Задание 2.
Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. 

In [76]:
document = ''.join(str(combine_df['clean_tweet'].tolist()))

In [78]:
pos_tag = nltk.pos_tag(nltk.word_tokenize(document))

In [79]:
tag_fd = nltk.FreqDist(tag for (word, tag) in pos_tag)

In [81]:
tag_fd.most_common(20)

[('NN', 143986),
 ('JJ', 66735),
 ("''", 57280),
 ('IN', 52898),
 (',', 49158),
 ('NNS', 33514),
 ('RB', 30125),
 ('VB', 28701),
 ('PRP', 28088),
 ('VBP', 26485),
 ('DT', 25662),
 ('VBZ', 22238),
 ('VBG', 17537),
 ('TO', 15176),
 ('VBD', 12078),
 ('PRP$', 11127),
 ('POS', 10821),
 ('CC', 10204),
 ('VBN', 8637),
 ('MD', 7141)]

In [87]:
ne = nltk.ne_chunk(pos_tag)

In [90]:
# tag_ne = nltk.FreqDist(tag for (word, tag) in ne)

In [ ]:
# tag_ne.most_common(20)

Действительно ли в топ вошли только персоны и организации или есть мусор?

In [ ]:
Только мусор

Задание 3.
Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.


In [ ]:
Однозначно, Spacy лучше отрабатывает.